In [231]:
import pandas as pd
import re

In [276]:
file_1 = '1-1-2022 (1).xlsx'
file_2 = '2-1-2022 (1).xlsx'
file_3 = '3-1-2022.xlsx'

In [233]:
FULLWIDTH_TO_HALFWIDTH = str.maketrans('１２３４５６７８９０',
                                       '1234567890')
def fullwidth_to_halfwidth(s):
    return s.translate(FULLWIDTH_TO_HALFWIDTH)

In [229]:
def gen_basic(month, detCol, xlsxName=''):
    opMonth = pd.to_datetime(month+'.01', format='%Y.%m.%d').date()

    # load file_2 data
    df_2 = pd.read_excel(file_2, sheet_name=month, skiprows=3, header=None)
    df_2 = df_2[:df_2[df_2[0] == '合計'].index.values[0]]
    df_2.fillna(0, inplace=True)

    # load file_1 data
    df_1 = pd.read_excel(file_1, sheet_name=month, skiprows=4, header=None)
    df_1 = df_1[:df_1[df_1[0] == '合　計'].index.values[0]]

    df_1.fillna(0, inplace=True)

    # process file 1 header
    df_1_header = pd.read_excel(file_1, sheet_name=month, header=None, nrows=4)
    str_dt = fullwidth_to_halfwidth(df_1_header.iloc[0, 43])

    m = re.match('(\d+)\D+(\d+)\D+(\d+)\D+', str_dt)
    publishDate_1 = pd.to_datetime(
        '-'.join(list(m.groups())), format='%Y-%m-%d').date()

    # fill horizontally for na
    df_1_header.fillna(method='ffill', axis=1, inplace=True)
    df_1_header.fillna(method='ffill', inplace=True)  # fill vertically for na
    df_1_header = df_1_header[1:]

    # for merged cells
    df_1_header.iloc[1, 30] = df_1_header.iloc[0, 30]
    df_1_header.iloc[1, 31] = df_1_header.iloc[0, 31]

    df_1_header.iloc[1, 44] = df_1_header.iloc[0, 44]
    df_1_header.iloc[1, 45] = df_1_header.iloc[0, 45]

    merged_cols = list(range(8, 30))+list(range(32, 44))

    tech_head = df_1_header.iloc[0, merged_cols] + \
        '_' + df_1_header.iloc[1, merged_cols]
    df_1_header = df_1_header.append(tech_head, ignore_index=True)
    df_1_header.rename(index={2: 4}, inplace=True)
    df_1_header.sort_index(inplace=True)
    df_1_header.fillna(method='ffill', inplace=True)  # fill vertically for na

    lst_dicts = []
    for ridx, row in df_1.iterrows():
        companyName = ''
        # print(row)
        if row[detCol] != '○':
            continue
        companyName = row[0]
        j = 8  # for df_2 col index
        for i in range(8, 48, 2):
            d1 = {}
            d1['OpMonth'] = opMonth
            d1['CompanyNameLocal'] = companyName
            d1['Technology'] = df_1_header.iloc[2, i]
            d1['NumberOfPlant'] = row[i]
            d1['MaxOutputMW'] = row[i+1]
            d1['GenMW'] = df_2.iloc[ridx, j]
            d1['PublishDate'] = publishDate_1
            lst_dicts.append(d1)
            j += 1
        # break

    df_result = pd.DataFrame(lst_dicts)
    if xlsxName:
        df_result.to_excel('output/{}.xlsx'.format(xlsxName), index=False)

    return df_result


In [279]:
def gen_7(month, xlsxName=''):
    opMonth = pd.to_datetime(month+'.01', format='%Y.%m.%d').date()
    
    df_3 = pd.read_excel(file_3, sheet_name=month, skiprows=6, header=None)
    cols = list(range(0, 29)) + list(range(37, df_3.shape[0]))
    df_3 = df_3.iloc[cols]
    df_3.reset_index(drop=True, inplace=True)
    df_3 = df_3[:df_3[df_3[0]=='合　計'].index.values[0]]
    df_3.fillna(0, inplace=True)
    
    df_3_header = pd.read_excel(file_3, sheet_name=month, header=None, nrows=6)
    df_3_header.fillna(method='ffill', inplace=True)  # fill vertically for na
    
    str_dt = fullwidth_to_halfwidth(df_3_header.iloc[0, 16])

    m = re.match('(\d+)\D+(\d+)\D+(\d+)\D+', str_dt)
    publishDate_3 = pd.to_datetime(
        '-'.join(list(m.groups())), format='%Y-%m-%d').date()

    lst_dicts = []
    for ridx, row in df_3.iterrows():
        companyName = ''
        # print(row)
        companyName = row[0]
        
        str_bits = ''
        for i in range(1, 8):
            if row[i] == '○':
                str_bits += '1'
            else:
                str_bits += '0'
                
        for i in range(8, 20):
            d1 = {}
            d1['OpMonth'] = opMonth
            d1['CompanyName'] = companyName
            d1['CompanyType'] = str_bits
            d1['DemandType'] = df_3_header.iloc[5, i]
            d1['DemandMW'] = row[i]
            d1['PublishDate'] = publishDate_3
            lst_dicts.append(d1)
    df_result = pd.DataFrame(lst_dicts)
    
    if xlsxName:
        df_result.to_excel('output/{}.xlsx'.format(xlsxName), index=False)
    df_result

In [230]:
gen_basic('2022.10', 2)

C:\Users\zhy\AppData\Local\Temp/ipykernel_20052/755701181.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_header = df_1_header.append(tech_head, ignore_index=True)


,OpMonth,CompanyNameLocal,Technology,NumberOfPlant,MaxOutputMW,GenMW,PublishDate
0,2022-10-01,北海道電力ネットワーク株式会社,水力発電所_一般,3.0,415.0,77.0,2023-02-02
1,2022-10-01,北海道電力ネットワーク株式会社,水力発電所_揚水式,0.0,0.0,0.0,2023-02-02
2,2022-10-01,北海道電力ネットワーク株式会社,水力発電所_計,3.0,415.0,77.0,2023-02-02
3,2022-10-01,北海道電力ネットワーク株式会社,火力発電_石炭,0.0,0.0,0.0,2023-02-02
4,2022-10-01,北海道電力ネットワーク株式会社,火力発電_ＬＮＧ,0.0,0.0,0.0,2023-02-02
...,...,...,...,...,...,...,...
195,2022-10-01,沖縄電力株式会社,新エネルギー等発電所_〔バイオマス〕,0.0,0.0,2750.0,2023-02-02
196,2022-10-01,沖縄電力株式会社,新エネルギー等発電所_〔廃棄物〕,0.0,0.0,0.0,2023-02-02
197,2022-10-01,沖縄電力株式会社,新エネルギー等発電所_計,5.0,2315.0,116.0,2023-02-02
198,2022-10-01,沖縄電力株式会社,その他,0.0,0.0,0.0,2023-02-02


In [280]:
gen_7('2022.10', 'test')

# BACKUP, DO NOT EXECUTE THE FOLLOWING

In [234]:
month = '2022.10'

In [235]:
opMonth = pd.to_datetime(month+'.01', format='%Y.%m.%d').date()
opMonth

datetime.date(2022, 10, 1)

In [210]:
df_2 = pd.read_excel(file_2, sheet_name=month, skiprows=3, header=None)
df_2 = df_2[:df_2[df_2[0]=='合計'].index.values[0]]
df_2.fillna(0, inplace=True)
df_2

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,北海道電力株式会社,○,0,0,0.0,0,○,○,250461,45509.0,...,1177138.0,0.0,0.0,106,5870,0.0,0.0,5976,0,1479083
1,北海道電力ネットワーク株式会社,0,○,0,0.0,0,○,0,77,0.0,...,4275.0,0.0,0.0,0,0,0.0,0.0,0,0,4352
2,東北電力株式会社,○,0,0,0.0,0,○,○,339057,1576.0,...,4152184.0,0.0,0.0,432,41550,0.0,0.0,41982,0,4534799
3,東北電力ﾈｯﾄﾜｰｸ株式会社,0,○,0,0.0,0,○,0,57,0.0,...,15076.0,0.0,0.0,0,0,0.0,0.0,0,0,15133
4,東京電力ホールディングス株式会社,0,0,0,0.0,0,○,○,0,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,伊方エコ・パーク株式会社,0,0,0,0.0,0,○,0,0,0.0,...,0.0,0.0,4080.0,0,0,0.0,0.0,4080,0,4080
1479,ES NPV1 合同会社,0,0,0,0.0,0,○,0,0,0.0,...,0.0,0.0,0.0,2504.2,0,0.0,0.0,2504.2,0,2504.2
1480,ES NPV5合同会社,0,0,0,0.0,0,○,0,0,0.0,...,0.0,0.0,0.0,2452.7,0,0.0,0.0,2452.7,0,2452.7
1481,日本エコエナジー合同会社,0,0,0,0.0,0,○,0,0,0.0,...,0.0,0.0,0.0,3518.2,0,0.0,0.0,3518.2,0,3518.2


In [211]:
df_1 = pd.read_excel(file_1, sheet_name=month, skiprows=4, header=None)
df_1 = df_1[:df_1[df_1[0]=='合　計'].index.values[0]]

In [212]:
df_1

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,北海道電力株式会社,○,0,0,0.0,0,○,○,52.0,855970.0,...,0.0,0.0,0.0,0.0,2.0,26000.0,0.0,0.0,66.0,8369370.0
1,北海道電力ネットワーク株式会社,0,○,0,0.0,0,○,0,3.0,415.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,16145.0
2,東北電力株式会社,○,0,0,0.0,0,○,○,203.0,1985139.0,...,0.0,0.0,0.0,0.0,8.0,143600.0,0.0,0.0,223.0,16640079.0
3,東北電力ﾈｯﾄﾜｰｸ株式会社,0,○,0,0.0,0,○,0,4.0,998.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,75148.0
4,東京電力ホールディングス株式会社,0,0,0,0.0,0,○,○,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8212000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,伊方エコ・パーク株式会社,0,0,0,0.0,0,○,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,18000.0,0.0,0.0,1.0,18000.0
1479,ES NPV1 合同会社,0,0,0,0.0,0,○,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,24000.0,0.0,0.0,1.0,24000.0
1480,ES NPV5合同会社,0,0,0,0.0,0,○,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,17640.0,0.0,0.0,1.0,17640.0
1481,日本エコエナジー合同会社,0,0,0,0.0,0,○,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,25000.0,0.0,0.0,1.0,25000.0


In [213]:
# df_1[df_1[4].isna()]

In [214]:
df_1.fillna(0, inplace=True)

In [215]:
df_1_header = pd.read_excel(file_1, sheet_name=month, header=None, nrows=4)

In [216]:
# publishDate_1 = 
str_dt = fullwidth_to_halfwidth(df_1_header.iloc[0, 43])
str_dt

'2023年2月2日公表時点'

In [217]:
m = re.match('(\d+)\D+(\d+)\D+(\d+)\D+', str_dt)
publishDate_1 = pd.to_datetime('-'.join(list(m.groups())), format='%Y-%m-%d').date()
publishDate_1
# list(m.groups())

datetime.date(2023, 2, 2)

In [218]:
df_1_header.fillna(method='ffill', axis=1, inplace=True) # fill horizontally for na
df_1_header.fillna(method='ffill', inplace=True) # fill vertically for na
df_1_header = df_1_header[1:]
df_1_header

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
1,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,水力発電所,水力発電所,...,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,その他,その他,計,計
2,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,一般,一般,...,〔バイオマス〕,〔バイオマス〕,〔廃棄物〕,〔廃棄物〕,計,計,計,計,計,計
3,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,発電所数,最大出力,...,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力


In [219]:
df_1_header.iloc[1, 30] = df_1_header.iloc[0, 30]
df_1_header.iloc[1, 31] = df_1_header.iloc[0, 31]
df_1_header[[30, 31]]

,30,31
1,原子力発電所,原子力発電所
2,原子力発電所,原子力発電所
3,発電所数,最大出力


In [220]:
df_1_header.iloc[1, 44] = df_1_header.iloc[0, 44]
df_1_header.iloc[1, 45] = df_1_header.iloc[0, 45]
df_1_header[[44, 45]]

,44,45
1,その他,その他
2,その他,その他
3,発電所数,最大出力


In [221]:
df_1_header

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
1,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,水力発電所,水力発電所,...,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,その他,その他,計,計
2,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,一般,一般,...,〔バイオマス〕,〔バイオマス〕,〔廃棄物〕,〔廃棄物〕,計,計,その他,その他,計,計
3,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,発電所数,最大出力,...,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力


In [222]:
merged_cols = list(range(8, 30))+list(range(32, 44))

tech_head = df_1_header.iloc[0, merged_cols] + \
    '_' + df_1_header.iloc[1, merged_cols]
df_1_header = df_1_header.append(tech_head, ignore_index=True)
df_1_header.rename(index={2: 4}, inplace=True)
df_1_header.sort_index(inplace=True)
df_1_header.fillna(method='ffill', inplace=True)  # fill vertically for na
df_1_header


C:\Users\zhy\AppData\Local\Temp/ipykernel_20052/376644972.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_header = df_1_header.append(tech_head, ignore_index=True)


,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,水力発電所,水力発電所,...,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,新エネルギー等発電所,その他,その他,計,計
1,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,一般,一般,...,〔バイオマス〕,〔バイオマス〕,〔廃棄物〕,〔廃棄物〕,計,計,その他,その他,計,計
3,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,水力発電所_一般,水力発電所_一般,...,新エネルギー等発電所_〔バイオマス〕,新エネルギー等発電所_〔バイオマス〕,新エネルギー等発電所_〔廃棄物〕,新エネルギー等発電所_〔廃棄物〕,新エネルギー等発電所_計,新エネルギー等発電所_計,その他,その他,計,計
4,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,発電所数,最大出力,...,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力,発電所数,最大出力


In [223]:
lst_dicts = []
for ridx, row in df_1.iterrows():
    companyName = ''
    # print(row)
    if row[7] != '○':
        continue
    companyName = row[0]
    j = 8 # for df_2 col index
    for i in range(8, 48, 2):
        d1 = {}
        d1['OpMonth'] = opMonth
        d1['CompanyNameLocal'] = companyName
        d1['Technology'] = df_1_header.iloc[2, i]
        d1['NumberOfPlant'] = row[i]
        d1['MaxOutputMW'] = row[i+1]
        d1['GenMW'] = df_2.iloc[ridx, j]
        d1['PublishDate'] = publishDate_1
        lst_dicts.append(d1)
        j += 1
    # break

df_result = pd.DataFrame(lst_dicts)

In [224]:
df_result.to_excel('output/202210.xlsx', index=False)

# 7

In [251]:
file_3 = '3-1-2022.xlsx'

In [271]:
df_3 = pd.read_excel(file_3, sheet_name=month, skiprows=6, header=None)
cols = list(range(0, 29)) + list(range(37, df_3.shape[0]))
df_3 = df_3.iloc[cols]
df_3.reset_index(drop=True, inplace=True)
df_3 = df_3[:df_3[df_3[0]=='合　計'].index.values[0]]
df_3.fillna(0, inplace=True)
df_3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,北海道電力株式会社,○,0,0,0,0,○,○,1371326,262366,837511,271449,248141,23308,370431.0,307431.0,63000.0,0.0,0.0,1741757.0
1,北海道電力ネットワーク株式会社,0,○,0,0,0,○,0,1406,0,1026,380,248,132,0.0,0.0,0.0,45368.0,4691.0,51465.0
2,東北電力株式会社,○,0,0,0,0,○,○,4070771,1594174,1847923,628674,585411,43263,785604.0,641746.0,143858.0,0.0,0.0,4856375.0
3,東北電力ﾈｯﾄﾜｰｸ株式会社,0,○,0,0,0,○,0,3337,0,2340,997,0,997,0.0,0.0,0.0,117754.0,18615.0,139706.0
4,東京電力ホールディングス株式会社,0,0,0,0,0,○,○,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,東日本旅客鉄道株式会社,0,0,0,0,○,○,0,4688,4688,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
709,六本木ｴﾈﾙｷﾞｰｻｰﾋﾞｽ株式会社,0,0,0,0,○,0,0,7434,0,7137,297,297,0,0.0,0.0,0.0,0.0,0.0,0.0
710,住友共同電力株式会社,0,0,0,0,○,○,0,90,0,41,49,40,9,0.0,0.0,0.0,0.0,0.0,0.0
711,JFEスチール株式会社,0,0,0,0,○,○,0,2391,0,2375,16,0,16,0.0,0.0,0.0,0.0,0.0,0.0


In [272]:
df_3_header = pd.read_excel(file_3, sheet_name=month, header=None, nrows=6)
df_3_header.fillna(method='ffill', inplace=True)  # fill vertically for na
df_3_header

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,３－（１）．電力需要実績 （２０２２年１０月）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,２０２３年２月２日公表時点,NaN,NaN,NaN
1,○みなし小売電気事業者等,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,２０２３年２月２日公表時点,NaN,NaN,"（単位：1,000kWh）"
2,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,その他需要\n（自由料金）,NaN,NaN,NaN,NaN,NaN,特定需要\n（経過措置料金）,NaN,２０２３年２月２日公表時点,最終保障供給,離島供給,合 計
3,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,その他需要\n（自由料金）,NaN,NaN,NaN,NaN,NaN,特定需要\n（経過措置料金）,NaN,２０２３年２月２日公表時点,最終保障供給,離島供給,合 計
4,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,その他需要計,特別高圧,高圧,低圧計,NaN,NaN,特定需要計,電灯,電力,最終保障供給,離島供給,合 計
5,事業者名,小売電気事業者,一般送配電事業者,送電事業者,配電事業者,特定送配電事業者,発電事業者,特定卸供給事業者,その他需要計,特別高圧,高圧,低圧計,電灯,電力,特定需要計,電灯,電力,最終保障供給,離島供給,合 計


In [277]:
str_dt = fullwidth_to_halfwidth(df_3_header.iloc[0, 16])

m = re.match('(\d+)\D+(\d+)\D+(\d+)\D+', str_dt)
publishDate_1 = pd.to_datetime(
    '-'.join(list(m.groups())), format='%Y-%m-%d').date()
publishDate_1

datetime.date(2023, 2, 2)

In [273]:
df_3_header.iloc[5, 8]

'その他需要計'

In [278]:
lst_dicts = []
for ridx, row in df_3.iterrows():
    companyName = ''
    # print(row)
    companyName = row[0]
    
    str_bits = ''
    for i in range(1, 8):
        if row[i] == '○':
            str_bits += '1'
        else:
            str_bits += '0'
            
    for i in range(8, 20):
        d1 = {}
        d1['OpMonth'] = opMonth
        d1['CompanyName'] = companyName
        d1['CompanyType'] = str_bits
        d1['DemandType'] = df_3_header.iloc[5, i]
        d1['DemandMW'] = row[i]
        d1['PublishDate'] = publishDate_1
        lst_dicts.append(d1)
        
    # break
df_result = pd.DataFrame(lst_dicts)
df_result

,OpMonth,CompanyName,CompanyType,DemandType,DemandMW,PublishDate
0,2022-10-01,北海道電力株式会社,1000011,その他需要計,1371326,2023-02-02
1,2022-10-01,北海道電力株式会社,1000011,特別高圧,262366,2023-02-02
2,2022-10-01,北海道電力株式会社,1000011,高圧,837511,2023-02-02
3,2022-10-01,北海道電力株式会社,1000011,低圧計,271449,2023-02-02
4,2022-10-01,北海道電力株式会社,1000011,電灯,248141,2023-02-02
...,...,...,...,...,...,...
8551,2022-10-01,Daigasエナジー株式会社,0000100,電灯,0.0,2023-02-02
8552,2022-10-01,Daigasエナジー株式会社,0000100,電力,0.0,2023-02-02
8553,2022-10-01,Daigasエナジー株式会社,0000100,最終保障供給,0.0,2023-02-02
8554,2022-10-01,Daigasエナジー株式会社,0000100,離島供給,0.0,2023-02-02
